<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/quickstart_autokeras_tabledata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テーブルデータを使った分類モデルをauto-kerasで構築

In [1]:
%pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1
%pip install autokeras

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc1) to /tmp/pip-req-build-0afc5355
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-0afc5355
  Running command git checkout -q 0fb69434a132093518e0e53d40020145ae192629
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc1-cp36-none-any.whl size=85424 sha256=c591b8b3707cd1585f8b158086dc61015a78fa70c1855d3458a064f275605b56
  Stored in directory: /tmp/pip-ephem-wheel-cache-3f6vwgyn/wheels/af/c9/7c/6ea01f9753a5dd1484136b4cb7b33a0a7fba253e5c74ade5af
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=1725e6ea18f8a66c6a05547e211dc40bfd8c381bb1c18e59e67689e5cc5e91f9
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
     |████████████████████████████████| 122kB 2.8MB/s 


In [2]:
%pip show autokeras
%pip show keras-tuner
%pip show tensorflow

Name: autokeras
Version: 1.0.8
Summary: AutoML for deep learning
Home-page: http://autokeras.com
Author: Data Analytics at Texas A&M (DATA) Lab, Keras Team
Author-email: jhfjhfj1@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: scikit-learn, tensorflow, pandas, packaging
Required-by: 
Name: keras-tuner
Version: 1.0.2rc1
Summary: Hypertuner for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The Keras Tuner authors
Author-email: kerastuner@google.com
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, colorama, tqdm, tabulate, scipy, packaging, terminaltables, future, requests, scikit-learn
Required-by: 
Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, a

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/機械学習練習/AutoML/auto-keras
%ls

Mounted at /gdrive
/gdrive/My Drive/機械学習練習/AutoML/auto-keras
california_housing/                   structured_data_classifier/
dataset/                              titanic.ipynb
mnist.ipynb                           wine_table_data.ipynb
quickstart_autokeras_tabledata.ipynb


In [16]:
import os

import autokeras as ak
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.metrics import confusion_matrix

## 前準備

In [21]:
# データセットへのパス
DATA_DIR = 'dataset/wine-dataset'  #@param {type: "string"}
TRAIN_CSV = 'train.csv'  #@param {type: "string"}
TEST_CSV = 'test.csv'  #@param {type: "string"}

# 予測対象の列名
TARGET_COL = 'target' #@param {type: "string"}
NUM_CLASSES = 3 #@param {type: "number"}


# モデルのへのハイパーパラメータ
# 探索するモデルの上限数
MAX_TRIALS = 15  #@param {type: "slider", min: 0, max: 100}
# 各モデルのエポック数
EPOCHS = 50
SEED = 7
TASK_NAME = 'wine-classification'

In [32]:
# タスク(引数に渡すCSVファイル)の確認
# モデルのfit関数の引数にはpandas.Dataframe形式ではなく、ファイルのパスを指定するだけで良い
train_df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_CSV), header=0)
test_df = pd.read_csv(os.path.join(DATA_DIR, TEST_CSV), header=0)
print("予測対象の列名:{}".format(TARGET_COL))
print("train shape:{}".format(train_df.shape))
print("test shape:{}".format(test_df.shape))
display(train_df.head())

予測対象の列名:target
train shape:(142, 14)
test shape:(36, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,12.96,3.45,2.35,18.5,106,1.39,0.70,0.40,0.94,5.28,0.68,1.75,675,2
1,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2
2,14.12,1.48,2.32,16.8,95,2.20,2.43,0.26,1.57,5.00,1.17,2.82,1280,0
3,13.58,2.58,2.69,24.5,105,1.55,0.84,0.39,1.54,8.66,0.74,1.80,750,2
4,13.30,1.72,2.14,17.0,94,2.40,2.19,0.27,1.35,3.95,1.02,2.77,1285,0


## Auto-kerasでモデル作成

### モデルのクラス初期化

In [33]:
clf = ak.StructuredDataClassifier(max_trials=MAX_TRIALS,# 探索するモデルの上限数
                                  seed=7,
                                  directory=TASK_NAME,
                                  overwrite=True) 

### モデルの訓練（探索）

In [34]:
clf.fit(x=os.path.join(DATA_DIR, TRAIN_CSV),
        y=TARGET_COL,
        epochs=EPOCHS, # 各モデルで回すエポック数(default:1000, earlystoppingあり)
        callbacks=[EarlyStopping(monitor='val_loss',patience=2)], # kerasモデル同様にcallback処理を設定可能
        validation_split=0.2, # 訓練データの末尾から指定した割合が検証データとなる(default:0.2)
        verbose=1, # 通常のkerasモデルで使われるようなキーワード引数も渡せる
        )

Trial 15 Complete [00h 00m 07s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 02s
INFO:tensorflow:Oracle triggered exit
Epoch 1/50
5/5 [==============================] - 0s 2ms/step - loss: 1.1045 - accuracy: 0.5070
Epoch 2/50
5/5 [==============================] - 0s 2ms/step - loss: 0.8664 - accuracy: 0.7676
Epoch 3/50
5/5 [==============================] - 0s 2ms/step - loss: 0.6757 - accuracy: 0.9296
Epoch 4/50
5/5 [==============================] - 0s 2ms/step - loss: 0.5221 - accuracy: 0.9648
Epoch 5/50
5/5 [==============================] - 0s 2ms/step - loss: 0.4036 - accuracy: 0.9648
Epoch 6/50
5/5 [==============================] - 0s 2ms/step - loss: 0.3140 - accuracy: 0.9789
Epoch 7/50
5/5 [==============================] - 0s 2ms/step - loss: 0.2459 - accuracy: 0.9789
Epoch 8/50
5/5 [==============================] - 0s 2ms/step - loss: 0.1949 - accuracy: 0.9789
Epoch 9/50
5/5 [==============================] - 0s 2ms/step - loss: 0.1568 - ac

## モデルの評価

### テストデータに対する性能

In [35]:
result = clf.evaluate(x=os.path.join(DATA_DIR, TEST_CSV), 
                      y=TARGET_COL)
print('loss:{}, accuracy:{}'.format(result[0], result[1]))

2/2 [==============================] - 0s 2ms/step - loss: 0.0917 - accuracy: 0.9722
loss:0.09166794270277023, accuracy:0.9722222089767456


### 最良のモデルをエクスポート

In [36]:
best_model = clf.export_model()
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 13)                0         
_________________________________________________________________
normalization (Normalization (None, 13)                27        
_________________________________________________________________
dense (Dense)                (None, 128)               1792      
_________________________________________________________________
re_lu (ReLU)                 (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)               

### sklearnを使用し混同行列を表示

In [38]:
# 正解ラベルの読み込み
y_test = test_df[TARGET_COL].values
#モデルの予測結果
y_pred = clf.predict(x=os.path.join(DATA_DIR, TEST_CSV))
matrix = confusion_matrix(y_true=y_test, y_pred=y_pred)
display(matrix)

array([[ 7,  0,  0],
       [ 0, 17,  0],
       [ 0,  1, 11]])